In [2]:
!pip install opencv-python mediapipe numpy

In [12]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Initialize MediaPipe Hand module
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7, max_num_hands=2)

# Define gesture mappings for both hands
finger_gestures = {
    "00001_00001": "1",  # Pinky up on both hands
    "00011_00011": "2",  # Pinky + Ring up on both hands
    "00111_00111": "3",  # Pinky + Ring + Middle up on both hands
    "01111_01111": "4",  # Four fingers up on both hands
    "11111_11111": "5",  # All fingers up on both hands

    "11111_00000": "+",  # Left all up, Right all down
    "00000_11111": "-",  # Right all up, Left all down
    "11011_00000": "*",  # Left: Thumb, Index, Pinky up
    "00000_11011": "/",  # Right: Thumb, Index, Pinky up

    "00000_00000": "=",  # Both fists closed → Evaluate
}

expression = ""  # Store arithmetic expression
last_fingers = ""  # Track last detected gesture
last_eval_time = 0  # Track last evaluation time

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    left_fingers = "00000"  # Default for left hand
    right_fingers = "00000"  # Default for right hand

    if results.multi_hand_landmarks:
        for hand_index, hand_landmarks in enumerate(results.multi_hand_landmarks):
            landmarks = [(lm.x, lm.y) for lm in hand_landmarks.landmark]

            # Identify which fingers are up
            fingers = ""
            finger_tips = [8, 12, 16, 20]  # Index, Middle, Ring, Pinky
            for tip in finger_tips:
                if landmarks[tip][1] < landmarks[tip - 2][1]:  # If tip is above DIP joint
                    fingers += "1"
                else:
                    fingers += "0"

            # Thumb detection
            fingers = ("1" if landmarks[4][0] < landmarks[3][0] else "0") + fingers

            # Assign based on hand order
            if hand_index == 0:
                left_fingers = fingers
            else:
                right_fingers = fingers

        # Combine both hands' gestures
        combined_gesture = f"{left_fingers}_{right_fingers}"

        # Check if gesture is different and valid
        if combined_gesture != last_fingers and combined_gesture in finger_gestures:
            gesture = finger_gestures[combined_gesture]

            if gesture in "0123456789+-*/":
                expression += gesture  # Append numbers/operators
            elif gesture == "=":
                current_time = time.time()
                if current_time - last_eval_time > 1.5:  # Only evaluate once every 1.5 sec
                    if expression.strip():  # Avoid empty eval()
                        try:
                            expression = str(eval(expression))
                        except:
                            expression = "Error"
                    last_eval_time = current_time  # Update last eval time
            last_fingers = combined_gesture  # Update last detected pattern

        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    else:
        last_fingers = ""  # Reset when no hand is detected

    # Draw a black box for calculations
    cv2.rectangle(frame, (20, 20), (300, 100), (0, 0, 0), -1)  # Black box
    cv2.putText(frame, expression, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)

    cv2.imshow("Hand Calculator", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
